In [4]:
from __future__ import print_function
import os
import time
import logging
import argparse
import numpy as np
from visdom import Visdom
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from utils import *

# Teacher models:
# VGG11/VGG13/VGG16/VGG19, GoogLeNet, AlxNet, ResNet18, ResNet34, 
# ResNet50, ResNet101, ResNet152, ResNeXt29_2x64d, ResNeXt29_4x64d, 
# ResNeXt29_8x64d, ResNeXt29_32x64d, PreActResNet18, PreActResNet34, 
# PreActResNet50, PreActResNet101, PreActResNet152, 
# DenseNet121, DenseNet161, DenseNet169, DenseNet201, 
import models

# Student models:
# myNet, LeNet, FitNet

start_time = time.time()
# os.makedirs('./checkpoint', exist_ok=True)

# Training settings
parser = argparse.ArgumentParser(description='PyTorch DML 2S')

parser.add_argument('--dataset',
                    choices=['CIFAR10',
                             'CIFAR100'
                            ],
                    default='CIFAR10')
parser.add_argument('--net1',
                    choices=['ResNet8',
                             'ResNet15',
                             'ResNet16',
                             'ResNet20',
                             'ResNet32',
                             'ResNet50',
                             'ResNet56',
                             'ResNet110'
                            ],
                    default='ResNet20')
parser.add_argument('--net2',
                    choices=['ResNet8',
                             'ResNet15',
                             'ResNet16',
                             'ResNet20',
                             'ResNet32',
                             'ResNet50',
                             'ResNet56',
                             'ResNet110'
                            ],
                    default='ResNet20')

parser.add_argument('--n_class', type=int, default=10, metavar='N', help='num of classes')
parser.add_argument('--batch_size', type=int, default=128, metavar='N', help='input batch size for training')
parser.add_argument('--test_batch_size', type=int, default=128, metavar='N', help='input test batch size for training')
parser.add_argument('--epochs', type=int, default=20, metavar='N', help='number of epochs to train (default: 20)')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR', help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M', help='SGD momentum (default: 0.5)')
parser.add_argument('--device', default='cuda:1', type=str, help='device: cuda or cpu')
parser.add_argument('--print_freq', type=int, default=40, metavar='N', help='how many batches to wait before logging training status')

config = ['--epochs', '200', '--net1', 'ResNet8', '--net2', 'ResNet8', '--device', 'cuda:1']
args = parser.parse_args(config)

device = args.device if torch.cuda.is_available() else 'cpu'
save_dir = './checkpoint/' + args.dataset + '/'

# models
net1 = getattr(models, args.net1)(num_classes=args.n_class)
net1.to(device)
net2 = getattr(models, args.net2)(num_classes=args.n_class)
net2.to(device)

# logging
logfile = save_dir + 'DML_2S_' + net1.model_name + '.log'
if os.path.exists(logfile):
    os.remove(logfile)
def log_out(info):
    f = open(logfile, mode='a')
    f.write(info)
    f.write('\n')
    f.close()
    print(info)
    
# visualizer
vis = Visdom(env='distill')
loss_win = vis.line(
    X=np.array([0]),
    Y=np.array([0]),
    opts=dict(
        title='DML Loss',
        xlabel='epoch',
        xtickmin=0,
        ylabel='loss',
        ytickmin=0,
        ytickstep=0.5,
        legend=['net1_loss', 'net2_loss']
    ),
    name="loss"
)

acc_win = vis.line(
    X=np.column_stack((0, 0)),
    Y=np.column_stack((0, 0)),
    opts=dict(
        title='DML Acc',
        xlabel='epoch',
        xtickmin=0,
#         xtickstep=5,
        ylabel='accuracy',
        ytickmin=0,
        ytickmax=100,
#         markers=True,
#         markersymbol='dot',
#         markersize=5,
        legend=['net1_acc', 'net2_acc']
    ),
    name="acc"
)


# data
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    normalize,
])
test_transform = transforms.Compose([transforms.ToTensor(), normalize])
train_set = datasets.CIFAR10(root='../data', train=True, download=True, transform=train_transform)
test_set = datasets.CIFAR10(root='../data', train=False, download=False, transform=test_transform)
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=args.test_batch_size, shuffle=False)

# optimizer = optim.SGD(st_model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer_1 = optim.SGD(net1.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
optimizer_2 = optim.SGD(net2.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
lr_scheduler_1 = optim.lr_scheduler.MultiStepLR(optimizer_1, milestones=[100, 150])
lr_scheduler_2 = optim.lr_scheduler.MultiStepLR(optimizer_2, milestones=[100, 150])

def DML_loss(y, labels, logits_1, logits_2):
    return nn.KLDivLoss()(logits_2, logits_1) + F.cross_entropy(y, labels)
    
# train with multi-teacher
def train(epoch, net1, net2):
    print('Training:')
    # switch to train mode
    net1.train()
    net2.train()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses_1 = AverageMeter()
    losses_2 = AverageMeter()
    top1_1 = AverageMeter()
    top1_2 = AverageMeter()
    
    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.to(device), target.to(device)
        
        # compute outputs
        _,_,_,_, output_1 = net1(input)
        _,_,_,_, output_2 = net2(input)
        logits_1 = F.softmax(output_1)
        logits_2 = F.softmax(output_2)
        
        optimizer_1.zero_grad()
        loss_1 = DML_loss(output_1, target, logits_1, logits_2)
        loss_1.backward(retain_graph=True)
        optimizer_1.step()
        
        optimizer_2.zero_grad()
        loss_2 = DML_loss(output_2, target, logits_2, logits_1)
        loss_2.backward(retain_graph=True)
        optimizer_2.step()

        output_1 = output_1.float()
        loss_1 = loss_1.float()
        # measure accuracy and record loss
        net1_acc = accuracy(output_1, target)[0]
        net2_acc = accuracy(output_2, target)[0]
        losses_1.update(loss_1.item(), input.size(0))
        top1_1.update(net1_acc, input.size(0))
        losses_2.update(loss_2.item(), input.size(0))
        top1_2.update(net2_acc, input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            log_out('[{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 ({top1_1.avg:.3f})'.format(
                      i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses_1, top1_1=top1_1))
    return losses_1.avg, losses_2.avg, net1_acc.cpu().numpy(), net2_acc.cpu().numpy()


def test(model):
    print('Testing:')
    # switch to evaluate mode
    model.eval()
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input, target = input.to(device), target.to(device)

            # compute output
            _,_,_,_,output = model(input)
            loss = F.cross_entropy(output, target)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            test_acc = accuracy(output.data, target.data)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(test_acc, input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                log_out('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(test_loader), batch_time=batch_time, loss=losses,
                          top1=top1))

    log_out(' * {0} Prec@1 {top1.avg:.3f}'.format(model.model_name, top1=top1))

    return losses.avg, test_acc.cpu().numpy(), top1.avg.cpu().numpy()


print('*-----------------DML----------------*')
best_acc = 0
for epoch in range(1, args.epochs + 1):
    log_out("\n===> epoch: {}/{}".format(epoch, args.epochs))
#     log_out('current lr {:.5e}'.format(optimizer_1.param_groups[0]['lr']))
    lr_scheduler_1.step()
    lr_scheduler_2.step()
    train_loss_1, train_loss_2, net1_acc, net2_acc = train(epoch, net1, net2)
    # visaulize loss
    vis.line(np.column_stack((train_loss_1, train_loss_2)), np.column_stack((epoch, epoch)), loss_win, update="append")
    _, test_acc_1, top1_1 = test(net1)
    _, test_acc_2, top1_2 = test(net2)
    vis.line(np.column_stack((top1_1, top1_2)), np.column_stack((epoch, epoch)), acc_win, update="append")
    best_acc = max(top1_1, top1_2, best_acc)

log_out("@ BEST Prec: {:.4f}".format(best_acc))
log_out("--- {:.3f} mins ---".format((time.time() - start_time)/60))


/home/data/yaliu/jupyterbooks/multi-KD/models/student/resnet_s.py:13: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)


Files already downloaded and verified
*-----------------DML----------------*

===> epoch: 1/200
Training:
[0/391]	Time 0.057 (0.057)	Data 0.019 (0.019)	Loss 2.2688 (2.2688)	Prec@1 (10.938)


/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:178: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:179: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/functional.py:1992: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


[40/391]	Time 0.051 (0.052)	Data 0.018 (0.018)	Loss 1.7698 (1.9370)	Prec@1 (18.674)
[80/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 1.5639 (1.8033)	Prec@1 (23.775)
[120/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 1.5513 (1.7226)	Prec@1 (27.014)
[160/391]	Time 0.052 (0.051)	Data 0.019 (0.018)	Loss 1.4284 (1.6573)	Prec@1 (29.746)
[200/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 1.3395 (1.6031)	Prec@1 (32.008)
[240/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 1.1944 (1.5540)	Prec@1 (34.112)
[280/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 1.3028 (1.5202)	Prec@1 (35.657)
[320/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 1.2896 (1.4846)	Prec@1 (37.191)
[360/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 1.2679 (1.4511)	Prec@1 (38.690)
Testing:
Test: [0/79]	Time 0.017 (0.017)	Loss 1.7067 (1.7067)	Prec@1 44.531 (44.531)
Test: [40/79]	Time 0.017 (0.017)	Loss 1.9192 (1.7234)	Prec@1 35.156 (43.083)
 * ResNet8 Prec@1 42.370
Testing:
Test: [0/79]	Time 0.017 (0.017)	Loss 1.99

Testing:
Test: [0/79]	Time 0.018 (0.018)	Loss 1.1559 (1.1559)	Prec@1 59.375 (59.375)
Test: [40/79]	Time 0.018 (0.017)	Loss 1.1381 (1.1310)	Prec@1 60.156 (61.643)
 * ResNet8 Prec@1 61.330
Testing:
Test: [0/79]	Time 0.017 (0.017)	Loss 1.0789 (1.0789)	Prec@1 64.062 (64.062)
Test: [40/79]	Time 0.016 (0.017)	Loss 1.1719 (1.0656)	Prec@1 58.594 (63.053)
 * ResNet8 Prec@1 63.010

===> epoch: 8/200
Training:
[0/391]	Time 0.052 (0.052)	Data 0.019 (0.019)	Loss 0.5770 (0.5770)	Prec@1 (76.562)
[40/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 0.4954 (0.6369)	Prec@1 (72.828)
[80/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 0.6666 (0.6495)	Prec@1 (72.637)
[120/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 0.6475 (0.6408)	Prec@1 (72.998)
[160/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 0.7608 (0.6391)	Prec@1 (73.137)
[200/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 0.6128 (0.6380)	Prec@1 (73.197)
[240/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 0.6996 (0.6378)	Prec@1 (73.117)
[280/3

...

Testing:
Test: [0/79]	Time 0.021 (0.021)	Loss 0.3121 (0.3121)	Prec@1 90.625 (90.625)
Test: [40/79]	Time 0.021 (0.021)	Loss 0.4276 (0.3653)	Prec@1 87.500 (88.434)
 * ResNet8 Prec@1 88.140
Testing:
Test: [0/79]	Time 0.022 (0.022)	Loss 0.2364 (0.2364)	Prec@1 91.406 (91.406)
Test: [40/79]	Time 0.019 (0.020)	Loss 0.4932 (0.3691)	Prec@1 83.594 (87.843)
 * ResNet8 Prec@1 87.760

===> epoch: 173/200
Training:
[0/391]	Time 0.105 (0.105)	Data 0.022 (0.022)	Loss 0.1112 (0.1112)	Prec@1 (92.969)
[40/391]	Time 0.106 (0.103)	Data 0.020 (0.020)	Loss 0.0891 (0.1185)	Prec@1 (92.245)
[80/391]	Time 0.103 (0.102)	Data 0.020 (0.020)	Loss 0.0830 (0.1222)	Prec@1 (91.917)
[120/391]	Time 0.103 (0.102)	Data 0.020 (0.020)	Loss 0.1237 (0.1220)	Prec@1 (92.000)
[160/391]	Time 0.106 (0.102)	Data 0.020 (0.020)	Loss 0.1321 (0.1224)	Prec@1 (92.047)
[200/391]	Time 0.104 (0.103)	Data 0.020 (0.020)	Loss 0.0771 (0.1234)	Prec@1 (92.020)
[240/391]	Time 0.106 (0.103)	Data 0.019 (0.020)	Loss 0.0663 (0.1257)	Prec@1 (91.941)
[280

[160/391]	Time 0.097 (0.103)	Data 0.019 (0.020)	Loss 0.0273 (0.1216)	Prec@1 (91.979)
[200/391]	Time 0.110 (0.103)	Data 0.026 (0.020)	Loss 0.0643 (0.1181)	Prec@1 (92.125)
[240/391]	Time 0.099 (0.103)	Data 0.023 (0.020)	Loss 0.0684 (0.1193)	Prec@1 (92.132)
[280/391]	Time 0.099 (0.103)	Data 0.021 (0.020)	Loss 0.1663 (0.1190)	Prec@1 (92.157)
[320/391]	Time 0.098 (0.103)	Data 0.020 (0.020)	Loss 0.0865 (0.1189)	Prec@1 (92.146)
[360/391]	Time 0.097 (0.103)	Data 0.020 (0.020)	Loss 0.1215 (0.1194)	Prec@1 (92.133)
Testing:
Test: [0/79]	Time 0.025 (0.025)	Loss 0.3333 (0.3333)	Prec@1 89.062 (89.062)
Test: [40/79]	Time 0.021 (0.022)	Loss 0.4253 (0.3651)	Prec@1 85.938 (88.453)
 * ResNet8 Prec@1 88.470
Testing:
Test: [0/79]	Time 0.020 (0.020)	Loss 0.2458 (0.2458)	Prec@1 89.844 (89.844)
Test: [40/79]	Time 0.020 (0.021)	Loss 0.4438 (0.3658)	Prec@1 85.938 (87.729)
 * ResNet8 Prec@1 87.750

===> epoch: 180/200
Training:
[0/391]	Time 0.106 (0.106)	Data 0.021 (0.021)	Loss 0.1574 (0.1574)	Prec@1 (91.406)
[4

Test: [40/79]	Time 0.020 (0.020)	Loss 0.4775 (0.3779)	Prec@1 84.375 (87.367)
 * ResNet8 Prec@1 87.610

===> epoch: 186/200
Training:
[0/391]	Time 0.104 (0.104)	Data 0.022 (0.022)	Loss 0.1847 (0.1847)	Prec@1 (90.625)
[40/391]	Time 0.104 (0.099)	Data 0.020 (0.020)	Loss 0.1176 (0.1155)	Prec@1 (92.454)
[80/391]	Time 0.103 (0.100)	Data 0.021 (0.020)	Loss 0.1361 (0.1187)	Prec@1 (92.139)
[120/391]	Time 0.121 (0.101)	Data 0.021 (0.021)	Loss 0.0959 (0.1154)	Prec@1 (92.284)
[160/391]	Time 0.106 (0.102)	Data 0.020 (0.020)	Loss 0.2145 (0.1143)	Prec@1 (92.357)
[200/391]	Time 0.084 (0.100)	Data 0.021 (0.020)	Loss 0.0375 (0.1147)	Prec@1 (92.339)
[240/391]	Time 0.073 (0.097)	Data 0.020 (0.020)	Loss 0.1116 (0.1156)	Prec@1 (92.317)
[280/391]	Time 0.098 (0.093)	Data 0.020 (0.020)	Loss 0.0384 (0.1161)	Prec@1 (92.249)
[320/391]	Time 0.098 (0.095)	Data 0.021 (0.020)	Loss 0.1712 (0.1159)	Prec@1 (92.229)
[360/391]	Time 0.097 (0.095)	Data 0.020 (0.020)	Loss 0.1369 (0.1165)	Prec@1 (92.205)
Testing:
Test: [0/79]

[280/391]	Time 0.105 (0.101)	Data 0.019 (0.019)	Loss 0.1036 (0.1117)	Prec@1 (92.499)
[320/391]	Time 0.106 (0.101)	Data 0.019 (0.019)	Loss 0.0722 (0.1132)	Prec@1 (92.475)
[360/391]	Time 0.104 (0.101)	Data 0.019 (0.019)	Loss 0.1047 (0.1120)	Prec@1 (92.532)
Testing:
Test: [0/79]	Time 0.025 (0.025)	Loss 0.3663 (0.3663)	Prec@1 85.938 (85.938)
Test: [40/79]	Time 0.019 (0.020)	Loss 0.4589 (0.3804)	Prec@1 86.719 (87.729)
 * ResNet8 Prec@1 87.670
Testing:
Test: [0/79]	Time 0.020 (0.020)	Loss 0.2798 (0.2798)	Prec@1 89.062 (89.062)
Test: [40/79]	Time 0.020 (0.020)	Loss 0.4406 (0.3775)	Prec@1 84.375 (87.195)
 * ResNet8 Prec@1 87.370

===> epoch: 193/200
Training:
[0/391]	Time 0.078 (0.078)	Data 0.020 (0.020)	Loss 0.1086 (0.1086)	Prec@1 (91.406)
[40/391]	Time 0.095 (0.102)	Data 0.019 (0.020)	Loss 0.1427 (0.1061)	Prec@1 (92.721)
[80/391]	Time 0.096 (0.102)	Data 0.019 (0.019)	Loss 0.0929 (0.1040)	Prec@1 (92.901)
[120/391]	Time 0.096 (0.102)	Data 0.022 (0.019)	Loss 0.1618 (0.1049)	Prec@1 (92.891)
[160

[40/391]	Time 0.103 (0.102)	Data 0.020 (0.021)	Loss 0.1630 (0.0995)	Prec@1 (93.083)
[80/391]	Time 0.106 (0.103)	Data 0.019 (0.021)	Loss 0.0759 (0.1056)	Prec@1 (92.747)
[120/391]	Time 0.104 (0.101)	Data 0.019 (0.021)	Loss 0.0842 (0.1098)	Prec@1 (92.549)
[160/391]	Time 0.098 (0.101)	Data 0.019 (0.020)	Loss 0.0700 (0.1095)	Prec@1 (92.615)
[200/391]	Time 0.110 (0.102)	Data 0.024 (0.020)	Loss 0.0946 (0.1099)	Prec@1 (92.642)
[240/391]	Time 0.104 (0.102)	Data 0.019 (0.021)	Loss 0.1288 (0.1088)	Prec@1 (92.706)
[280/391]	Time 0.106 (0.102)	Data 0.022 (0.021)	Loss 0.1814 (0.1096)	Prec@1 (92.657)
[320/391]	Time 0.105 (0.102)	Data 0.020 (0.020)	Loss 0.0850 (0.1088)	Prec@1 (92.669)
[360/391]	Time 0.095 (0.102)	Data 0.019 (0.021)	Loss 0.0896 (0.1095)	Prec@1 (92.661)
Testing:
Test: [0/79]	Time 0.139 (0.139)	Loss 0.3883 (0.3883)	Prec@1 86.719 (86.719)
Test: [40/79]	Time 0.020 (0.029)	Loss 0.4719 (0.3747)	Prec@1 87.500 (87.843)
 * ResNet8 Prec@1 87.940
Testing:
Test: [0/79]	Time 0.029 (0.029)	Loss 0.28